In [1]:
import pandas as pd
import summary_functions as sf
import arcpy
from arcgis import GeoAccessor
from arcgis.features import FeatureLayer
import os

sample_year = 2022
dataset = 'acs/acs5'

In [2]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'

feature_layer = FeatureLayer(service_url)
tahoe_geometry_fields = ['YEAR', 'STATE', 'GEOGRAPHY', 'GEOID', 'TRPAID', 'NEIGHBORHOOD']
query_result = feature_layer.query(out_fields=",".join(tahoe_geometry_fields))
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
tahoe_geometry = pd.DataFrame([feature.attributes for feature in feature_list])

county_lookup = {
    '005': 'Douglas County (Tahoe Basin)',
    '017': 'El Dorado County (Tahoe Basin)',
    '031': 'Washoe County (Tahoe Basin)',
    '061': 'Placer County (Tahoe Basin)'
}

state_lookup = {
    '32': 'Nevada',
    '06': 'California'
}

inflation_adjustment = {
    '2000': 1.57,
    '2010': 1.24
}

service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
all_data = pd.DataFrame([feature.attributes for feature in feature_list])

all_data_clean = all_data.loc[(all_data['county']!='510')&(all_data['value']>=0)]
all_data_clean.loc[all_data_clean['county'].isin(['005','017']),'north_south'] = 'South Lake'
all_data_clean.loc[all_data_clean['county'].isin(['031','061']),'north_south'] = 'North Lake'
all_data_clean['county_name'] = all_data_clean['county'].apply(lambda x: county_lookup.get(x, None))
all_data_clean['state_name'] = all_data_clean['state'].apply(lambda x: state_lookup.get(x, None))


all_data_clean = all_data_clean.dropna(subset='value')

C:\Users\amcclary\AppData\Local\Temp\ipykernel_20984\1107904247.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_clean.loc[all_data_clean['county'].isin(['005','017']),'north_south'] = 'South Lake'
C:\Users\amcclary\AppData\Local\Temp\ipykernel_20984\1107904247.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_clean['county_name'] = all_data_clean['county'].apply(lambda x: county_lookup.get(x, None))
C:\Users\amcclary\AppData\Local\Temp\ipykernel_20984\1107904247.py:43: SettingWithC

In [5]:
filtered_df=all_data_clean.loc[(all_data_clean['dataset']==dataset)]

summary_df = sf.sum_across_levels_moe(filtered_df,'Total Population','Total Population')

summary_df.to_excel("../Census_Data_Downloads/Population_Totals_ACS.xlsx")